# 04-02 : Function Hub

Attempt a function from MLRun Function Hub.

> ❗️❗️PLEASE NOTE: The credentials created during the setup are a problem. Rather do the following now to resolve this issue.

```bash
# log into docker locally
docker login -u shambi

# delete the existing secret
sudo kubectl --namespace mlrun delete secret registry-credentials

# create a new secret
sudo kubectl --namespace mlrun create secret docker-registry registry-credentials \
    --from-file=/home/johnny/.docker/config.json
```


## References

- https://docs.mlrun.org/en/latest/runtimes/image-build.html

In [1]:
import os

## 1. Configuration 

In [2]:
# set the mlrun api path
mlrun_api_ip = !(hostname -I | awk '{print $1}')
mlrun_api_ip = mlrun_api_ip[0]
mlrun_api_port = 30070

os.environ["MLRUN_DBPATH"] = f"http://{mlrun_api_ip}:{mlrun_api_port}"
print(f"MLRUN_DBPATH: {os.environ['MLRUN_DBPATH']}")

MLRUN_DBPATH: http://192.168.1.184:30070


In [3]:
import mlrun

## 2. Create Project

In [4]:
project = mlrun.get_or_create_project(
    name='hello-project',
    context='./',) 

> 2025-05-23 14:14:50,771 [info] Created and saved project: {"context":"./","from_template":null,"name":"hello-project","overwrite":false,"save":true}
> 2025-05-23 14:14:50,772 [info] Project created successfully: {"project_name":"hello-project","stored_in_db":true}


## 3. Get Describe function from Hub

In [5]:
project.set_function("hub://describe", "describe")
project.save()

In [6]:
my_describe = project.get_function("describe")

In [7]:
my_describe.doc()

function: describe
describe and visualizes dataset stats
default handler: analyze
entry points:
  analyze: The function will output the following artifacts per
column within the data frame (based on data types)
If the data has more than 500,000 sample we
sample randomly 500,000 samples:

describe csv
histograms
scatter-2d
violin chart
correlation-matrix chart
correlation-matrix csv
imbalance pie chart
imbalance-weights-vec csv
    context(MLClientCtx)  - The function context
    name(str)  - Key of dataset to database ("dataset" for default), default=dataset
    table(Union[FeatureSet, DataItem])  - MLRun input pointing to pandas dataframe (csv/parquet file path) or FeatureSet as param, default=None
    label_column(str)  - Ground truth column label, default=None
    plots_dest(str)  - Destination folder of summary plots (relative to artifact_path) ("plots" for default), default=plots
    random_state(int)  - When the table has more than 500,000 samples, we sample randomly 500,000 samp

## 4. Run the function

In [8]:
run = my_describe.run(
    name="describe",
    inputs={"table": "https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv"},
    params={"label_column": "label"},
)

> 2025-05-23 14:14:51,158 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2025-05-23 14:14:51,170 [info] Storing function: {"db":"http://192.168.1.184:30070","name":"describe","uid":"1111fa009e3d4af19c6ed7408753743b"}
> 2025-05-23 14:14:51,248 [info] Job is running in the background, pod: describe-zt268
> 2025-05-23 12:14:56,355 [info] The data set is logged to the project under dataset name
> 2025-05-23 12:14:56,413 [info] To track results use the CLI: {"info_cmd":"mlrun get run 1111fa009e3d4af19c6ed7408753743b -p hello-project","logs_cmd":"mlrun logs 1111fa009e3d4af19c6ed7408753743b -p hello-project"}
> 2025-05-23 12:14:56,413 [info] Run execution finished: {"name":"describe","status":"completed"}


project,uid,iter,start,state,kind,name,labels,inputs,parameters,results,artifacts
hello-project,...53743b,0,May 23 12:14:52,completed,run,describe,kind=jobowner=johnnymlrun/client_version=1.7.2mlrun/client_python_version=3.9.22host=describe-zt268,table,label_column=label,,describe-csvhisthistogramsscatter-2dviolinimbalanceimbalance-weights-veccorrelation-matrix-csvcorrelationdataset


> 2025-05-23 14:15:02,311 [info] Run execution finished: {"name":"describe","status":"completed"}


In [9]:
result = project.build_function(
    "describe",
    with_mlrun=False,
    force_build=True,
    skip_deployed=False,
    overwrite_build_params=False,
    )
result

> 2025-05-23 14:15:04,229 [info] Started building image: .shambi/func-hello-project-describe:latest


The `overwrite_build_params` parameter default will change from 'False' to 'True' in 1.8.0.


INFO[0001] Retrieving image manifest mlrun/mlrun:1.7.2  
INFO[0001] Retrieving image mlrun/mlrun:1.7.2 from registry index.docker.io 
INFO[0003] Built cross stage deps: map[]                
INFO[0003] Retrieving image manifest mlrun/mlrun:1.7.2  
INFO[0003] Returning cached image manifest              
INFO[0003] Executing 0 build triggers                   
INFO[0003] Building stage 'mlrun/mlrun:1.7.2' [idx: '0', base-idx: '-1'] 
INFO[0003] Skipping unpacking as no commands require it. 
INFO[0003] Pushing image to index.docker.io/shambi/func-hello-project-describe:latest 
INFO[0011] Pushed index.docker.io/shambi/func-hello-project-describe@sha256:3ddb8506edd11f080cd41309bfee2e8ad153395b910f60c00e2493addfe8ef49 


BuildStatus(ready=True, outputs={'image': '.shambi/func-hello-project-describe:latest'})

In [10]:
result

BuildStatus(ready=True, outputs={'image': '.shambi/func-hello-project-describe:latest'})

In [11]:
?project.build_image

Signature:
project.build_image(
    image: str = None,
    set_as_default: bool = True,
    with_mlrun: bool = None,
    base_image: str = None,
    commands: list = None,
    secret_name: str = None,
    requirements: Union[str, list[str]] = None,
    mlrun_version_specifier: str = None,
    builder_env: dict = None,
    overwrite_build_params: bool = False,
    requirements_file: str = None,
    extra_args: str = None,
    target_dir: str = None,
) -> Union[mlrun.projects.operations.BuildStatus, kfp.dsl._container_op.ContainerOp]
Docstring:
Builder docker image for the project, based on the project's build config. Parameters allow to override
the build config.
If the project has a source configured and pull_at_runtime is not configured, this source will be cloned to the
image built. The `target_dir` parameter allows specifying the target path where the code will be extracted.

:param image: target image name/path. If not specified the project's existing `default_image` name will be
 

In [12]:
project

In [13]:
project.build_image(
    target_dir='./',
    with_mlrun=True)

> 2025-05-23 14:26:34,071 [info] Base image not specified, using default base image: {"base_image":"mlrun/mlrun"}
> 2025-05-23 14:26:34,073 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2025-05-23 14:26:34,075 [info] Running build to add mlrun package, set with_mlrun=False to skip if its already in the image
> 2025-05-23 14:26:34,138 [info] Started building image: .mlrun-project-image-hello-project


The `overwrite_build_params` parameter default will change from 'False' to 'True' in 1.8.0.


INFO[0001] Retrieving image manifest mlrun/mlrun:1.7.2  
INFO[0001] Retrieving image mlrun/mlrun:1.7.2 from registry index.docker.io 
INFO[0003] Built cross stage deps: map[]                
INFO[0003] Retrieving image manifest mlrun/mlrun:1.7.2  
INFO[0003] Returning cached image manifest              
INFO[0003] Executing 0 build triggers                   
INFO[0003] Building stage 'mlrun/mlrun:1.7.2' [idx: '0', base-idx: '-1'] 
INFO[0003] Skipping unpacking as no commands require it. 
INFO[0003] Pushing image to index.docker.io/shambi/mlrun-project-image-hello-project 
INFO[0012] Pushed index.docker.io/shambi/mlrun-project-image-hello-project@sha256:fe5d843f9d179179b7f195466131438ece6bfe5dbedab42b70c5bd9a55826fb9 
> 2025-05-23 14:26:54,454 [info] Function is being deleted: {"function_name":"mlrun--project--image--builder","project_name":"hello-project"}
> 2025-05-23 14:26:57,473 [info] Function deleted: {"function_name":"mlrun--project--image--builder","project_name":"hello-project

BuildStatus(ready=True, outputs={'image': '.mlrun-project-image-hello-project'})